In [28]:
import pandas as pd
import numpy as np

In [29]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [30]:
#BASE = 'afastamentos'
BASE = 'cadastro'

In [31]:
df = pd.read_csv(BASE + '-nomes-sexo.csv', index_col=0, 
                 dtype={'PERC': np.float32, 'LETRAS': 'uint8'})

/home/rodrigo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
df.drop(columns=['REV', 'QTDE'], inplace=True)

In [33]:
df.head(10)

,PERC,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,,
MARIA,3.1593,F,5,A,I,R,A,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JOSE,2.4671,M,4,E,S,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANA,1.7812,F,3,A,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JOAO,1.1643,M,4,O,A,O,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARLOS,1.1418,M,6,S,O,L,R,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAULO,1.1186,M,5,O,L,U,A,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANTONIO,1.0817,M,7,O,I,N,O,T,N,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LUIZ,0.9511,M,4,Z,I,U,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FRANCISCO,0.8978,M,9,O,C,S,I,C,N,A,R,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df['SEXO'] = df['SEXO'].map({'F': 0, 'M': 1})
df.loc[df['SEXO'].isnull(), 'SEXO'] = 2
df['SEXO'] = df['SEXO'].astype('uint8')

In [35]:
for i in range(16):
    col = 'L' + str(i)
    df.loc[df[col].isnull(), col] = ' '
    df[col] = df[col].apply(lambda x: ord(x) - 64 if x != ' ' else 0).astype('uint8')

In [36]:
df.dtypes

PERC      float32
SEXO        uint8
LETRAS      uint8
L0          uint8
L1          uint8
L2          uint8
L3          uint8
L4          uint8
L5          uint8
L6          uint8
L7          uint8
L8          uint8
L9          uint8
L10         uint8
L11         uint8
L12         uint8
L13         uint8
L14         uint8
L15         uint8
dtype: object

In [37]:
df.head(10)

,PERC,SEXO,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,,
MARIA,3.1593,0,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,2.4671,1,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,1.7812,0,3,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1.1643,1,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1.1418,1,6,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0
PAULO,1.1186,1,5,15,12,21,1,16,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1.0817,1,7,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
LUIZ,0.9511,1,4,26,9,21,12,0,0,0,0,0,0,0,0,0,0,0,0
FRANCISCO,0.8978,1,9,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0


In [38]:
#df.describe()

In [39]:
# definir dados de entrada

X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (48970, 18) (48970,)


In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
#rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe = RFECV(estimator=DecisionTreeClassifier())
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe), ('m',model)])

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores)*100, np.std(n_scores)*100))

rfe.fit(X, y)

# summarize all features
for i in range(X.shape[1]):
  print('Column: %d [%s], Rank: %.3f %s' % (
      i, X.columns[i], rfe.ranking_[i],
      'Selected ' if rfe.support_[i] else ''))

Accuracy: 99.379 (0.086)
Column: 0 [PERC], Rank: 1.000 Selected 
Column: 1 [LETRAS], Rank: 3.000 
Column: 2 [L0], Rank: 1.000 Selected 
Column: 3 [L1], Rank: 1.000 Selected 
Column: 4 [L2], Rank: 1.000 Selected 
Column: 5 [L3], Rank: 2.000 
Column: 6 [L4], Rank: 4.000 
Column: 7 [L5], Rank: 5.000 
Column: 8 [L6], Rank: 6.000 
Column: 9 [L7], Rank: 7.000 
Column: 10 [L8], Rank: 13.000 
Column: 11 [L9], Rank: 12.000 
Column: 12 [L10], Rank: 8.000 
Column: 13 [L11], Rank: 11.000 
Column: 14 [L12], Rank: 10.000 
Column: 15 [L13], Rank: 9.000 
Column: 16 [L14], Rank: 14.000 
Column: 17 [L15], Rank: 15.000 


In [41]:
#df = df[['PERC', 'L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'SEXO']]
#df.drop(['QTDE'], axis=1, inplace=True)

In [42]:
X.head()

,PERC,LETRAS,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,
MARIA,3.1593,5,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,2.4671,4,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,1.7812,3,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1.1643,4,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1.1418,6,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [43]:
y.head()

PNOME
MARIA     0
JOSE      1
ANA       0
JOAO      1
CARLOS    1
Name: SEXO, dtype: uint8

In [44]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [45]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [46]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [47]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, multi_class='auto', max_iter=1000, solver='liblinear', C=0.01) # 85.18
evaluate_model('LR', model)

params = dict(solver=['liblinear','lbfgs'], C=np.logspace(-3,3,7))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(C=0.01, max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 79.27 (+/- 0.26) [ 4517 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.5s finished


In [48]:
# Support Vector Machine (SVM)
from sklearn.svm import SVC

model = SVC(random_state=42, C=10, gamma=0.001, kernel='rbf') # 89.49
evaluate_model('SVM', model)

#params = dict(C=np.logspace(-1,1,3)) # 87.90
#params = dict(gamma=np.logspace(-3,0,4)) # 89.46
#params = dict(kernel=['linear','rbf','poly','sigmoid']) # 89.46
params = dict(C=np.logspace(-1,1,3),
              gamma=np.logspace(-3,0,4),
              kernel=['linear','rbf','poly','sigmoid'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SVC(C=10, gamma=0.001, random_state=42) 
Cross-Validation Score: 86.42 (+/- 0.19) [355805 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.9min finished


In [49]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(C=np.logspace(-4,3,8))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 79.34 (+/- 0.20) [12827 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.8s finished


In [50]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1) # 87.45
evaluate_model('KNN', model)

params = dict(n_neighbors=[1,3,5,7,9,11])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(n_neighbors=1) 
Cross-Validation Score: 84.35 (+/- 0.23) [11651 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.6s finished


In [51]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(solver=['svd','lsqr','eigen'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 78.91 (+/- 0.12) [ 1067 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.1s finished


In [52]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 44.70 (+/- 0.17) [  452 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


In [53]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=0.1) # 67.62
evaluate_model('GNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(var_smoothing=0.1) 
Cross-Validation Score: 81.14 (+/- 0.12) [  362 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


In [54]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 43.68 (+/- 0.32) [  377 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


In [55]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17) # 96.37
evaluate_model('DT', model)

params = dict(criterion=['gini','entropy'], max_depth=[3,5,7,11,13,17,19])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(criterion='entropy', max_depth=17, random_state=42) 
Cross-Validation Score: 99.36 (+/- 0.07) [  751 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


In [56]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, max_features='auto', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(n_estimators=[10,50,100,500], max_features=['auto','sqrt','log2'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/home/rodrigo/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomForestClassifier(n_estimators=500, random_state=42) 
Cross-Validation Score: 96.77 (+/- 0.12) [93711 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [57]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), n_estimators=13) # 96.41
evaluate_model('ABDT', model)

params = dict(n_estimators=[1,3,5,7,9,11,13])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=13) 
Cross-Validation Score: 99.39 (+/- 0.03) [  792 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [58]:
# Ensemble - Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42, max_depth=35, n_estimators=250, learning_rate=0.1) # 96.75
evaluate_model('GB', model)

params = dict(max_depth=[1,3,5,7,9], n_estimators=[5,50,250,500], learning_rate=[0.01,0.1,1,10,100])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GradientBoostingClassifier(max_depth=35, n_estimators=250, random_state=42) 
Cross-Validation Score: 99.36 (+/- 0.06) [889223 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 14.8min finished


In [59]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=500, tol=0.1) # 77.02
evaluate_model('SGD', model)

params = dict(max_iter=[100, 200, 350, 500, 1000], tol=[0.01, 0.1, 1.0])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(max_iter=500, random_state=42, tol=0.1) 
Cross-Validation Score: 72.00 (+/- 5.90) [ 2910 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.9s finished


In [60]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.001) # 75.58
evaluate_model('PCT', model)

params = dict(max_iter=[100, 200, 350, 500, 750, 1000], tol=[0.1, 0.01, 0.001])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(max_iter=500, random_state=42) 
Cross-Validation Score: 71.61 (+/- 8.19) [ 1239 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [61]:
# Multi-Layer Perceptron (MLP)
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, solver='lbfgs', alpha=1e-07, hidden_layer_sizes=(50, 100, 50)) # 88.67
evaluate_model('MLP', model)

params = dict(alpha=np.logspace(-9,-1,5), 
              solver=['lbfgs','sgd','adam'], 
              hidden_layer_sizes=[(50,50,50), (50,100,50), (100,)])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


MLPClassifier(alpha=1e-07, hidden_layer_sizes=(50, 100, 50), random_state=42,
              solver='lbfgs') 
Cross-Validation Score: 88.16 (+/- 0.57) [282420 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.7min finished


In [62]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=9, min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.7, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.7, gamma=0.1,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              validate_parameters=None, verbosity=None) 
Cross-Validation Score: 99.62 (+/- 0.06) [86713 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished


In [63]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(num_leaves=73, max_depth=24, learning_rate=0.5, feature_fraction=0.75) # 97.51
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LGBMClassifier(feature_fraction=0.75, learning_rate=0.5, max_depth=24,
               num_leaves=73) 
Cross-Validation Score: 99.62 (+/- 0.04) [20454 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.5s finished


(99.62425975086788, 0.041549908003628476, 20454)

In [64]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier() # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(iterations=[10,50,100,250,500,1000],
              learning_rate=[0.01,0.05,0.1,0.5],
              depth=range(1,11,2)) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 99.41 (+/- 0.08) [169100 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.8min finished


In [65]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [66]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
16,LGBM,99.624260,0.041550,20454,"LGBMClassifier(feature_fraction=0.75, learning..."
15,XGB,99.616091,0.059044,86713,"XGBClassifier(base_score=None, booster=None, c..."
17,CB,99.407801,0.084444,169100,<catboost.core.CatBoostClassifier object at 0x...
10,ABDT,99.393506,0.032029,792,AdaBoostClassifier(base_estimator=DecisionTree...
8,DT,99.362875,0.068523,751,"DecisionTreeClassifier(criterion='entropy', ma..."
11,GB,99.358791,0.056812,889223,"GradientBoostingClassifier(max_depth=35, n_est..."
9,RF,96.765367,0.115046,93711,"RandomForestClassifier(n_estimators=500, rando..."
14,MLP,88.162140,0.565199,282420,"MLPClassifier(alpha=1e-07, hidden_layer_sizes=..."
1,SVM,86.418215,0.186086,355805,"SVC(C=10, gamma=0.001, random_state=42)"
3,KNN,84.345518,0.227340,11651,KNeighborsClassifier(n_neighbors=1)


In [67]:
model = models['XGB'][0]
model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.7, gamma=0.1,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              validate_parameters=None, verbosity=None)

In [68]:
model.fit(X, y)

/home/rodrigo/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:37:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=2, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [69]:
y_pred = model.predict(X)
y_pred

array([0, 1, 0, ..., 1, 1, 1], dtype=uint8)

In [70]:
dados = X[['PERC']]
dados['real'] = y
dados['prev'] = y_pred
dados.head()

<ipython-input-70-daf3dd0687f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['real'] = y
<ipython-input-70-daf3dd0687f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados['prev'] = y_pred


,PERC,real,prev
PNOME,,,
MARIA,3.1593,0,0
JOSE,2.4671,1,1
ANA,1.7812,0,0
JOAO,1.1643,1,1
CARLOS,1.1418,1,1


In [71]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

array([[21891,     0,     0],
       [    0, 19247,     0],
       [    0,     0,  7832]])

In [72]:
dados[dados['real'] != dados['prev']]

,PERC,real,prev
PNOME,,,
